## Moving Average as Foreign Exchange Basic Trading Strategy


This exercise performs a back-testing for a MA strategy. As is well known, back-testing results can never be achieved in live trading, therefore, this strategy is impossible to replicate. Check the next file, "MA FX Strategy - Out of Sample Testing", for a feasible FX strategy.

In [20]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot
import pandas_datareader as web
import datetime as dt

In [2]:
import scipy.stats

1. Importing market data from Yahoo Finance.

In [3]:
CAD=web.DataReader('CADUSD=X', 'yahoo', start = '08/31/2010')
#sp500.head()
CAD.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2643 entries, 2010-08-30 to 2020-10-22
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       2643 non-null   float64
 1   Low        2643 non-null   float64
 2   Open       2643 non-null   float64
 3   Close      2643 non-null   float64
 4   Volume     2643 non-null   float64
 5   Adj Close  2643 non-null   float64
dtypes: float64(6)
memory usage: 144.5 KB


2. Ploting the data can be done with various packages.

In [21]:
#Using Plotly Graphical Objects
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=CAD.index, y=CAD.Close, name='CAD/USD'))
fig1.update_layout(title="CAD/USD", xaxis_type="date", yaxis_type="log")#?
iplot(fig1, show_link=False)

In [5]:
#Using Plotly Express
import plotly.express as px
fig2 = px.line(x=CAD.index, y=CAD.Close, title = 'CADUSD')
fig2

In [6]:
# moving from Pandas to Numpy
LongOnlyReturn = np.log(CAD['Close']/(CAD['Close']).shift(1))
LongOnly = np.exp(np.cumsum(LongOnlyReturn)) 

3. Defining the MA Strategy as a function of MA parameters i, j representing the sizes of the rolling MA windows.

In [7]:
def MAstrat(i,j, SD=0.00):
    MA1 = np.array(CAD['Close'].rolling(i).mean())
    MA2 = np.array(CAD['Close'].rolling(j).mean())
    buy = MA1-MA2
    
    babs = np.abs(buy)
    babs[np.isnan(babs)] = 0

    buy[babs < SD] = 0
    buy[np.isnan(buy)] = 0
    Long = np.sign(buy)

    MAreturn = Long[:-1]*np.array(LongOnlyReturn[1:])
    MAstrategy = np.exp(np.cumsum(MAreturn))


    MAcrit = np.sum(MAreturn)/np.std(MAreturn) #the criterion for choosing the MA parameters
#     print(np.std(MAreturn))
    return MAcrit, MAreturn, MAstrategy, MA1, MA2, Long


In [8]:
price = np.array(CAD['Close'])

def MAstrategy1(now, back, i,j, SD=0.00): 
    #Calculates the exposures over the whole period for the desired i and j
    #where i and j are the MA parameters
    
    MA1 = np.convolve(price, np.ones(i), 'valid')/i
    MA2 = np.convolve(price, np.ones(j), 'valid')/j
    
    realSize = min(MA1.shape[0], MA2.shape[0])
    
    #buy is realSize
    buy = MA1[(MA1.shape[0]-realSize):]-MA2[(MA2.shape[0]-realSize):]
    buy[np.abs(buy) < SD] = 0
    #Long is realSize
    Long = np.sign(buy)
    
    offset = max(i, j)-1

    MAreturn = Long[:-1]*np.array(LongOnlyReturn[offset+1:]) #check the indices
    MAstrategy = np.exp(np.cumsum(MAreturn))
    MAcrit = np.sum(MAreturn)/np.std(MAreturn) 
    #the maximization criterion 
    #sum or average (it doesn't matter) scaled by standard deviation
    return MAcrit, Long

MAcrit = MAstrat(1,20)[0]; print(MAcrit)
MAcrit = MAstrategy1(1,1,1,20)[0]; MAcrit

-64.32295943719488


-64.09138971981739

The two methods give the same optimization result. However, the second method should be faster, because it gets rid of the rolling mean calculation with Pandas. It also eliminates the need to delete _nans_.

4. Find the optimal i, j parameters by maximizing the risk-adjusted value of the investment.

In [9]:
MAmaxcrit = -np.Inf
n=25
MACritij = np.zeros((n-1,n-1))

for i in range(1, n):
    for j in range(i+1, n):
        MAcrit = MAstrat(i,j, 0.00)[0]
        MACritij[i-1,j-1] = MAcrit
        if MAcrit>MAmaxcrit:
            MAmaxcrit = MAcrit
            imax = i
            jmax = j

print(f'i={imax}, j={jmax}, Maximization criterion = {MAmaxcrit}')


i=8, j=11, Maximization criterion = 86.14386608859134


In [10]:
# There are various ways to plot

z = MACritij
fig = go.Figure(data=[go.Surface(z=z)])
fig.update_layout(title='MA Parameters - Maximization Criterion', 
                  scene_camera_eye=dict(x=1, y=-1.5, z=0.25),
                  autosize=False,
                  width=600, height=600,
                  margin=dict(l=65, r=50, b=65, t=90))
fig

In [11]:
z = MACritij
#sh_0, sh_1 = z.shape
x, y = np.linspace(0, 1, n-1), np.linspace(0, 1, n-1)
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='MA Parameters Heatmap', 
                  scene_camera_eye=dict(x=1, y=-1.5, z=0.25),
                  autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig

In [12]:
i = imax
j = jmax
MAcrit, MAreturn, MAstrategy, MA1, MA2, Long = MAstrat(i,j, 0.00)

5. Plotting the cumulative log-returns of the MA strategy and of holding long-only CAD versus USD.

In [13]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=CAD.index, y=LongOnly, name='Long-only'))
fig.add_trace(go.Scatter(x=CAD.index, y=MAstrategy, name='MA Strategy'))
fig.update_layout(title="MA Trading Strategy") 
fig

6. Plotting the two moving averages and the long-short regimes.

In [14]:
#to redo the function output

layout = go.Layout(title='MA Trading Strategy', yaxis=dict(), yaxis2=dict(overlaying='y', side='right'))
fig = go.Figure(layout=layout)
fig.add_trace(go.Scatter(x=CAD.index, y=CAD['Close'], name='CAD/USD', yaxis='y2'))
fig.add_trace(go.Scatter(x=CAD.index, y=MA1, name='MA1',yaxis='y2'))
fig.add_trace(go.Scatter(x=CAD.index, y=MA2, name='MA2',yaxis='y2'))
fig.add_trace(go.Scatter(x=CAD.index, y=Long, name='Regime', yaxis='y1'))
fig

7. Calculating basic descriptive statistics for the MA strategy log-returns.

In [15]:
MAreturn = MAreturn[~np.isnan(MAreturn)]

In [16]:
x = scipy.stats.describe(MAreturn)
y= dict(x._asdict())
y["min"] = y["minmax"][0]
y["max"] = y["minmax"][1]
y.pop("minmax");y

{'nobs': 2642,
 'mean': 0.00015440405588953683,
 'variance': 2.2433587476945322e-05,
 'skewness': 0.047643528141605766,
 'kurtosis': 1.4363076239661083,
 'min': -0.02149129208769311,
 'max': 0.02189155609072135}

In [17]:
plotarray=[go.Histogram(x=MAreturn, nbinsx=50)]
figlayout={'title':'Distribution of MA Strategy returns'}
fig = go.Figure(data=plotarray, layout=figlayout)
fig

There is a large concentration of daily returns at zero because there are many days with zero exposure to the foreign currency. Otherwise, the distribution appears slightly positively skewed and with fatter tails than the normal distribution. 

In [18]:
#current exposure:
Long[-1]

-1.0